# ゼロから作る Deep Learning 4 強化学習編 勉強ノート 第7章 〜ニューラルネットワークと Q 学習〜

現実の問題は今までに扱ってきたグリッドワールドの問題のように単純ではなく、 Q 関数をコンパクトな関数で近似することが必要になる。  
そのための最も有力な手法がディープラーニングである。

## 線形回帰

機械学習は「データ」を使って問題を解く。  
その中でも最も基本となる「線形回帰」を実装する。

### トイ・データセット(*コーディング*)

実験用に小さなデータセットを作る。  
そのような小さなデータセットを<font color="red">**トイ・データセット**</font>(Toy Dataset)と呼ぶ。  
再現性を考慮して乱数のシードを固定してデータを生成する。

In [ ]:
!pip install -U japanize-matplotlib

In [ ]:
# ライブラリのインポート
import japanize_matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# シードを固定して乱数を発生
seed = 8192
np.random.seed(seed)

# トイ・データセットの作成
x = np.random.rand(100, 1)
y = 5 + 2 * x + np.random.rand(100, 1)

上記のように、 `x` と `y` の2つの変数からなるデータセットを作る。  
ここで、点群は直線上にあり、 `y` にはノイズとして乱数が加算されている。

In [ ]:
# Matplotlib の初期設定
plt.figure(figsize=(10, 6), tight_layout=True)
plt.title("トイ・データセット", size=15, color="red")
plt.xlabel("x")
plt.ylabel("y")
plt.grid()

# トイ・データセットの点を散布図で描画
plt.scatter(x, y, label="トイ・データセットの点", color="blue")

# 凡例を追加
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left", borderaxespad=0.)

# 描画
plt.show()

図を見ると `x` と `y` は「線形」の関係にあるが、そこにはノイズが含まれている。  
ここでの目標は `x` の値から `y` の値を予測するモデルを作ることである。

### 線形回帰の理論

データが与えられたとき、それに適合するような関数を見つけることが目標になる。  
ここでは、 $y$ と $x$ の関係は線形であると仮定しているため、 $y = Wx + b$ という式で表せる。  
一方で、データが完全にこの式に適合する訳ではないため、データと式から導かれる予測値との差をできる限り減らすことが求められている。  
この差を「残差」といい、モデルの予測値とデータが「どれほど適合していないか」を表す指標にこの残差を用いて次のような式で定義する。

$$L = \frac{1}{N} \sum_{i=1}^N (Wx_i + b - y_i)^2$$

全部で $N$ 個の点があるとして、 $(x_i, y_i)$ の各点において2乗した誤差を求め、それらを足し合わせる。  
その平均を求めるために $N$ で割っている。  
これを<font color="red">**平均2乗誤差**</font>(Mean Squared Error)という。  
我々の目標は、この損失関数が最小となる $W$ と $b$ を見つけることであり、すなわち関数の最適化問題である。

### 線形回帰の実装(*コーディング*)

ここでは線形回帰を実装する。

In [ ]:
# ライブラリのインポート
import torch
from torch import nn
import torch.optim as optim

In [ ]:
# PyTorch で作った線形回帰モデル
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [ ]:
# シードを固定して乱数を発生
seed = 8192
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

# トイ・データセットの作成
x = np.random.rand(100, 1)
y = 5 + 2 * x + np.random.rand(100, 1)

# Numpy 配列を PyTorch のテンソルに変換
x_train = torch.tensor(x, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.float32)

# モデルのインスタンスを生成
model = LinearRegressionModel()

# 損失関数(平均2乗誤差)と最適化アルゴリズム(SGD)の設定
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 1000エポックで学習
num_epochs = 1000
for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# 学習後のパラメータ(重みとバイアス)
[W, b] = model.parameters()
print("=====================")
print(f"学習後の重み: {W.item():.4f}")
print(f"学習後のバイアス: {b.item():.4f}")

損失関数の出力値が徐々に減っており、最終的には $W = 2, b = 5$ に近づいていることが確認できる。

In [ ]:
# Matplotlib の初期設定
plt.figure(figsize=(10, 6), tight_layout=True)
plt.title("トイ・データセット", size=15, color="red")
plt.xlabel("x")
plt.ylabel("y")
plt.grid()

# トイ・データセットの点を散布図で描画
plt.scatter(x, y, label="トイ・データセットの点", color="blue")

# 学習されたモデルのパラメータを取得
w_value = W.item()
b_value = b.item()

# 学習された直線を描画
x_line = np.linspace(0, 1, 100)
y_line = w_value * x_line + b_value
plt.plot(x_line, y_line, label="学習された直線", color="green", linewidth=2)

# 凡例を追加
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left", borderaxespad=0.)

# 描画
plt.show()

上図の通り、データに適合したモデルを得ることができた。

## ニューラルネットワーク

線形回帰が実装出来たので、同じように PyTorch を使ってニューラルネットワークを実装する。

### 非線形なデータセット(*コーディング*)

前節では直線上に並ぶデータセットを使った。  
ここでは正弦関数を用いてデータを生成する。

In [ ]:
# シードを固定して乱数を発生
seed = 8192
np.random.seed(seed)

# トイ・データセットの作成
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

# Matplotlib の初期設定
plt.figure(figsize=(10, 6), tight_layout=True)
plt.title("非線形トイ・データセット", size=15, color="red")
plt.xlabel("x")
plt.ylabel("y")
plt.grid()

# トイ・データセットの点を散布図で描画
plt.scatter(x, y, label="トイ・データセットの点", color="blue")

# 凡例を追加
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left", borderaxespad=0.)

# 描画
plt.show()

図の通り、 `x` と `y` は線形の関係ではない。  
このような非線形なデータセットにはもちろん線形回帰は対応していない。  
そこでニューラルネットワークを使う。

### 線形変換と活性化関数

線形回帰で行った計算は、損失関数を除くと「行列の積」と「足し算」だけであった。  
入力 `x` とパラメータ `W` との間で行列の積を求め、それに `b` を足し合わせた。  
この変換は、<font color="red">**線形変換**</font>(Linear Transformation)や<font color="red">**アフィン変換**</font>(Affine Transformation)と呼ばれる。  
線形変換は厳密には `b` の足し算を含まないが、ニューラルネットワークの分野ではここまでの演算を線形変換と呼ぶのが一般的である。  
また、線形変換はニューラルネットワークにおいては<font color="red">**全結合層**</font>に対応する。  
パラメータの `W` は<font color="red">**重み**</font>(Weight)、パラメータの `b` は<font color="red">**バイアス**</font>(Bias)と呼ばれる。

線形変換は、入力データに対して線形な変換を行うのに対し、ニューラルネットワークでは、線形変換の出力に対して非線形な変換を行う。  
その非線形な変換を行う関数を活性化関数と呼ぶ。  
代表的なものにはシグモイド関数や ReLU 関数などがある。

![シグモイド関数](https://resources.zero2one.jp/2022/05/ai_exp_87-1-768x432.jpeg)

(出典: https://zero2one.jp/ai-word/sigmoid-function/)

![ReLU 関数](https://mathlandscape.com/wp-content/uploads/2022/05/relu-new-1536x884.png)

(出典: https://mathlandscape.com/relu/)

それぞれのグラフが示すように、これらの活性化関数は非線形の関数である。  
ニューラルネットワークでは、このような非線形な変換がテンソルの要素ごとに適用される。

### ニューラルネットワークの実装(*コーディング*)

一般的なニューラルネットワークは「線形変換」と「活性化関数」を交互に使用する。  
これらを順に入力に対して適用することで、ニューラルネットワークの推論を可能にする。

In [ ]:
# PyTorch で作ったニューラルネットワークモデル
class NeuralNetworkModel(nn.Module):
    def __init__(self):
        super(NeuralNetworkModel, self).__init__()
        self.linear1 = nn.Linear(1, 10)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(10, 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.sigmoid(x)
        x = self.linear2(x)
        return x

In [ ]:
# シードを固定して乱数を発生
seed = 8192
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

# トイ・データセットの作成
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

# Numpy 配列を PyTorch のテンソルに変換
x_train = torch.tensor(x, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.float32)

# モデルのインスタンスを生成
model = NeuralNetworkModel()

# 損失関数(平均2乗誤差)と最適化アルゴリズム(SGD)の設定
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.2)

# 10000エポックで学習
num_epochs = 10000
for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 1000 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Matplotlib の初期設定
plt.figure(figsize=(10, 6), tight_layout=True)
plt.title("非線形トイ・データセット", size=15, color="red")
plt.xlabel("x")
plt.ylabel("y")
plt.grid()

# トイ・データセットの点を散布図で描画
plt.scatter(x, y, label="トイ・データセットの点", color="blue")

# 学習された曲線を描画
x_eval = torch.tensor(
    [[data] for data in np.linspace(0, 1, 100)], dtype=torch.float32
)
model.eval()
with torch.no_grad():
    y_pred = model(x_eval).detach().numpy()
x_eval = x_eval.detach().numpy()
plt.plot(x_eval, y_pred, label="学習された曲線", color="green", linewidth=2)

# 凡例を追加
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left", borderaxespad=0.)

# 描画
plt.show()

正弦関数の曲線が上手く表現できている。

## Q 学習とニューラルネットワーク

前章では TD 法、中でも Q 学習という強化学習において最も有名なアルゴリズムについて学んだ。  
ここでのテーマは、 Q 学習とニューラルネットワークを「融合」させることである。  
まずはニューラルネットワークの前処理について扱う。

### ニューラルネットワークの前処理(*コーディング*)

「3×4のグリッドワールド」の問題では、状態が(0, 0)や(0, 1)のように表された。  
この状態はエージェントの位置が $(y, x)$ のデータ形式で表現されており、12通りある「カテゴリデータ」と考えることができる。  
これらを one-hot ベクトルへ変換する。

In [ ]:
# カテゴリデータを one-hot ベクトルに変換する関数を定義
def one_hot(state: set) -> list:
    HEIGHT, WIDTH = 3, 4
    vec = np.zeros(HEIGHT * WIDTH, dtype=np.float32)
    y, x = state
    idx = WIDTH * y + x
    vec[idx] = 1.0
    return vec[np.newaxis, :]

In [ ]:
# 状態を one-hot ベクトルへ変換する
state = (2, 0)
x = one_hot(state)
print(f"形式: {x.shape}")
print(f"変換後: {x}")

### Q 関数を表すニューラルネットワーク(*コーディング*)

これまで Q 関数を次のようにテーブルとして実装してきた。

In [ ]:
# ライブラリのインポート
from collections import defaultdict

In [ ]:
# Q 関数の辞書
Q = defaultdict(lambda: 0)
state = (2, 0)
action = 0
print(Q[state, action])

この Q 関数をニューラルネットワークへと変身させる。  
それにはまず、ニューラルネットワークの入出力をはっきりさせる必要がある。  
状態だけを入力として、行動の候補の数だけ Q 関数の値を出力するネットワークを考える。  
例えば、行動の候補が4つであれば4つの要素を持つベクトルを出力する。

In [ ]:
# Q 関数のニューラルネットワークモデル
class QNet(nn.Module):
    def __init__(self):
        super(QNet, self).__init__()
        self.linear1 = nn.Linear(12, 100)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(100, 4)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

In [ ]:
# モデルのインスタンスを生成
qnet = QNet()

# 状態を one-hot ベクトルへ変換する
state = (2, 0)
state = one_hot(state)
state = torch.tensor(state, dtype=torch.float32)

# Q 関数のニューラルネットワークに値を入れる
qs = qnet(state)
print(qs.shape)

これで Q 関数をニューラルネットワークに置き換えることができた。

### ニューラルネットワークと Q 学習(*コーディング*)

ここでは Q 学習のアルゴリズムを実装する。  
Q 学習では次の式によって Q 関数を更新する。

$$Q'(S_t, A_t) = Q(S_t, A_t) + \alpha \left \{ R_t + \gamma \max_a Q(S_{t+1}, a) - Q(S_t, A_t) \right \}$$

この式によって $Q(S_t, A_t)$ の値は $R_t + \gamma \max_a Q(S_{t+1}, a)$ の方向へと更新される。  
ここでターゲットである $R_t + \gamma \max_a Q(S_{t+1}, a)$ を $T$ とすると、

$$Q'(S_t, A_t) = Q(S_t, A_t) + \alpha \left \{ T - Q(S_t, A_t) \right \}$$

と表される。  
これは、入力が $S_t, A_t$ のとき出力が $T$ となるように Q 関数を更新すると解釈できる。  
これをニューラルネットワークの文脈に当てはめると、入力が $S_t, A_t$ で、出力が $T$ となるように学習させることと同じである。  
つまり $T$ はスカラ値の正解ラベルと見做せ、回帰問題と捉えることができる。

In [ ]:
# QLearningAgent クラスの実装
class QLearningAgent:
    def __init__(self):
        self.gamma = 0.9
        self.lr = 0.01
        self.eps = 0.1
        self.action_size = 4

        self.qnet = QNet()
        self.optimizer = optim.SGD(self.qnet.parameters(), lr=self.lr)
        self.criterion = nn.MSELoss()

    def get_action(self, state):
        if np.random.rand() < self.eps:
            return np.random.choice(self.action_size)
        else:
            qs = self.qnet(state)
            return torch.argmax(qs).item()

    def update(self, state, action, reward, next_state, done):
        done = int(done)
        state = torch.tensor(state, dtype=torch.float32)
        next_state = torch.tensor(next_state, dtype=torch.float32)
        reward = torch.tensor([reward], dtype=torch.float32)

        qs = self.qnet(state)
        q_value = qs[0, action]

        with torch.no_grad():
            next_qs = self.qnet(next_state)
            next_q_value = next_qs.max()
            target = reward + (1 - done) * self.gamma * next_q_value

        loss = self.criterion(q_value, target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

In [ ]:
# GridWorld クラスの実装
class GridWorld:
    def __init__(self):
        self.action_space = [0, 1, 2, 3]
        self.action_meaning = {
            0: "UP",
            1: "DOWN",
            2: "LEFT",
            3: "RIGHT"
        }
        self.reward_map = np.array(
            [
                [0, 0, 0, 1.0],
                [0, None, 0, -1.0],
                [0, 0, 0, 0]
            ]
        )
        self.goal_state = (0, 3)
        self.wall_state = (1, 1)
        self.start_state = (2, 0)
        self.agent_state = self.start_state

    @property
    def height(self):
        return len(self.reward_map)

    @property
    def width(self):
        return len(self.reward_map[0])

    @property
    def shape(self):
        return self.reward_map.shape

    def actions(self):
        return self.action_space

    def states(self):
        for h in range(self.height):
            for w in range(self.width):
                yield (h, w)

    def next_state(self, state, action):
        action_move_map = [
            (-1, 0), (1, 0), (0, -1), (0, 1)
        ]
        move = action_move_map[action]
        next_state = (state[0] + move[0], state[1] + move[1])
        ny, nx = next_state

        if nx < 0 or nx >= self.width or ny < 0 or ny >= self.height:
            next_state = state
        elif next_state == self.wall_state:
            next_state = state

        return next_state

    def reward(self, state, action, next_state):
        return self.reward_map[next_state]

    def step(self, action):
        state = self.agent_state
        next_state = self.next_state(state, action)
        reward = self.reward(state, action, next_state)
        done = (next_state == self.goal_state)

        self.agent_state = next_state

        return next_state, reward, done

    def reset(self):
        self.agent_state = self.start_state
        return self.agent_state

In [ ]:
# ロスをプロットする関数を定義
def loss_show(history: list) -> None:
    x = [i for i in range(1, len(history) + 1)]

    plt.figure(figsize=(8, 6), tight_layout=True)
    plt.title("ロスの遷移", size=15, color="red")
    plt.grid()
    plt.plot(x, history)

In [ ]:
# 各クラスのインスタンスを生成
env = GridWorld()
agent = QLearningAgent()

# 1000回のエピソードで実行
episodes = 1000
loss_history = []
for episode in range(episodes):
    state = env.reset()
    state = one_hot(state)
    state = torch.tensor(state, dtype=torch.float32)
    total_loss, cnt = 0, 0
    done = False

    while not done:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)
        next_state = one_hot(next_state)
        next_state = torch.tensor(next_state, dtype=torch.float32)

        loss = agent.update(
            state, action, reward, next_state, done
        )
        total_loss += loss
        cnt += 1
        state = next_state

    average_loss = total_loss / cnt
    loss_history.append(average_loss)

# ロスの遷移画像を描画
loss_show(loss_history)

ニューラルネットワークを使った強化学習では、損失をプロットしても安定した結果が得られないことが多々ある。  
変化の振れ幅は大きいが、大きな視点ではエピソードを重ねるごとにロスが小さくなっていることが確認できる。